In [1]:
import pandas as pd

## LLM lib

from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from ollama import chat
import openpyxl

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
# messages_path = 'data\\AR_historic_messages.csv'
# conversations_path = 'data\\AR_historic_conversations_filtered.csv'

In [4]:
bot_messages_path = 'C:\\Users\\choc-\\Downloads\\01_Whapbot_Piloto_Responses.xlsx'

In [5]:
# messages_df = pd.read_csv(messages_path)
# conversations_df = pd.read_csv(conversations_path)

In [6]:
messages_df = pd.read_excel(bot_messages_path, sheet_name='Hoja1')

In [7]:
messages_df.head()

,MobileNumber,ActivityName,JourneyName,subscriberkey,eventdate,Status,message_hcp_reply,converstation_category,conversation_intention,conversation_need
0,524433356300,81_Piloto,01_MEX_XBU_Wappbot_1.0_Piloto_202402,MEX_AKA_24761b3d-f341-46cb-8d6a-a23dbe3e2c23,Mar 25 2024 3:22PM,DELIVERED,"""Hola""",NaN,NaN,NaN
1,524433714608,81_Piloto,01_MEX_XBU_Wappbot_1.0_Piloto_202402,MEX_AKA_6c07156b-1cc6-41ce-9558-1d55b0f56a5f,Mar 25 2024 3:22PM,DELIVERED,"""Hola""",NaN,NaN,NaN
2,524921361292,81_Piloto,01_MEX_XBU_Wappbot_1.0_Piloto_202402,0034P00003wzefQQAQ,Mar 25 2024 3:22PM,DELIVERED,"""Hola""",NaN,NaN,NaN
3,527441050781,81_Piloto,01_MEX_XBU_Wappbot_1.0_Piloto_202402,MEX_AKA_2356ca15-60e4-489b-b049-a0c42c7a80fe,Mar 25 2024 3:47PM,READ,"""HOLA""",NaN,NaN,NaN
4,525561387430,81_Piloto,01_MEX_XBU_Wappbot_1.0_Piloto_202402,0034P00003wzLPXQA2,Mar 25 2024 3:22PM,DELIVERED,"""Hola"".",NaN,NaN,NaN


In [ ]:
# conversations_id = conversations_df['conversation_id']

## LLM inference

In [ ]:
promt_template = """
    Eres un analizador de mensajes que responde a campañas o contenidos enviados. Tu tarea es procesar cada mensaje y devolver la siguiente información:

    1. Tipo de mensaje:

        - 1: Mensajes que son respuestas preestablecidas en las que se anuncian o presentan los servicios ofrecidos.

        - 0: Mensajes en los que el usuario agradece, pregunta o solicita ayuda de manera personalizada.

    2. Sentimiento (solo para mensajes escritos):

        - 2: Si el mensaje expresa agradecimiento, satisfacción u otra intención positiva.

        - 1: Si el mensaje muestra quejas, molestia o insatisfacción.

        - 0: Si el mensaje es neutral, sin emociones claras.

    3. Ayuda solicitada (solo para mensajes escritos):

        -Identifica qué tipo de asistencia solicita el usuario (por ejemplo, solicitar que un representante se comunique, pedir más información, requerir entradas adicionales para un evento, etc.)

    4. Tipo de ayuda solicitada (solo para mensajes escritos):
        - 1: Necesita que alguna persona se comunique con él o ella.

        - 0: Necesita ayuda técnica para resolver un problema (por ejemplo, problemas de acceso a una plataforma, no le deja consultar cierta información, etc.)

    El mensaje a analizar es el siguiente:
"""
message = """Hola, gracias por comunicarse con el Dr. Francisco Treviño.

Escriba el número de la opción deseada:
*1*. *Costos* de consulta
*2*. Agendar una *cita presencial*
*3*. Escribir una *opinión*
*4*. Emitir una *factura*
*5*. *Ubicación* de consultorios
*6*. """

In [ ]:
class Classification(BaseModel):
    Clasificacion: str = Field(description="Tipo de respuesta: (Automática/Escrita)")
    Sentitimiento: str = Field(
        description="Inteción: (Positiva/Negativa)"
    )
    Resumen: str = Field(
        description="sResumen breve de la conversación"
    )
    Necesidad: str = Field(
        description="Necesidad del usuario")

In [11]:
def call_llm(message_final):
    response = chat(
    messages=[
        {
        'role': 'user',
        'content': promt_template + message_final,
        }
    ],
    model='gemma3:12b',
    format=Classification.model_json_schema(),
    )

    country = Classification.model_validate_json(response.message.content)

    # print(country)
    return country.Clasificacion, country.Sentitimiento, country.Resumen, country.Necesidad

### Mensaje automático

In [9]:
message = """Hola, gracias por comunicarse con el Dr. Francisco Treviño.

Escriba el número de la opción deseada:
*1*. *Costos* de consulta
*2*. Agendar una *cita presencial*
*3*. Escribir una *opinión*
*4*. Emitir una *factura*
*5*. *Ubicación* de consultorios
*6*. """

In [10]:
call_llm(message)

Clasificacion='Automatico' Sentitimiento='N/A' Resumen='Mensaje automatizado que presenta opciones de consulta para los servicios del Dr. Francisco Treviño.' Necesidad='N/A'


### Mensaje Escrito

In [11]:
message = """Quiero al representante presencial, solicito apoyo de eventos y creo que de esta manera no me tomarán en cuenta.
Ya no viene nadie del laboratorio Sanofi a mi consultorio
Y tengo tiempo solicitando"""

In [12]:
call_llm(message)

Clasificacion='Escrito' Sentitimiento='Negativo' Resumen='El usuario expresa frustración por la falta de apoyo presencial, solicita asistencia con eventos y manifiesta preocupación de no ser considerado debido a la falta de comunicación.' Necesidad='Solicita comunicación con un representante presencial y apoyo para eventos.'


In [12]:
messages = messages_df['message_hcp_reply'].tolist()

In [18]:
len(messages)

323

In [13]:
category_list = []
feeling_list = []
conversation_list = []
need_list = []

In [19]:
for message in messages:
    # try:
    #     messages_text = messages_df[messages_df['conversation_id'] == id].sort_values(by='message_number')['message'].astype(str).tolist()
    #     if not messages_text:
    #         raise ValueError("messages_text is an empty list")
    # except Exception as e:
    #     print(f"Error processing conversation_id {id}: {e}")
    #     messages_text = []
    # final_message = '\n'.join(messages_text) if len(messages_text) > 0 else ''
    
    final_message = message.replace('"', '')
    classification = ''
    feeling = ''
    resume = ''
    need = ''

    if final_message != '':
        classification, feeling, resume, need = call_llm(final_message)
    
    category_list.append(classification)
    feeling_list.append(feeling)
    conversation_list.append(final_message)
    need_list.append(need)
    print('Conversacion leida')

Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion leida
Conversacion

"## 

In [21]:
with open('output.txt', 'w', encoding='utf-8') as file:
    for category, feeling, conversation, need in zip(category_list, feeling_list, conversation_list, need_list):
        file.write(f"{category}\t{feeling}\t{conversation}\t{need}\n")

In [20]:
messages_df['conversation_category'] = category_list
messages_df['conversation_intention'] = feeling_list
# messages['conversation'] = conversation_list
messages_df['conversation_need'] = need_list

ValueError: Length of values (333) does not match length of index (323)

In [ ]:
messages_df.to_excel('data/conversations_table.xlsx', index=False)